In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, join, QTable, vstack
import astropy.units as u
import sys
import pyneb as pn
from multiprocessing import Pool
import multiprocessing as mp
import math
from astropy.io import fits
from orcs.process import SpectralCube

from astropy.nddata import NDData, Cutout2D
from astropy.wcs import WCS

import astropy.units as u
from astropy.coordinates import SkyCoord

from reproject import reproject_interp
from regions import PixCoord

import pylab as pl

from scipy.optimize import curve_fit
from scipy.integrate import quad

from orb.fit import fit_lines_in_spectrum
from orb.utils.spectrum import corr2theta, amp_ratio_from_flux_ratio
from orb.core import Lines
import gvar
import orb

import extinction
from extinction import apply, remove

### Import data

In [ ]:
galdic = {1:'NGC4254', 2:'NGC4535', 3:'NGC3351', 4:'NGC2835', 5:'NGC0628'}  ## There is no SITELLE data for NGC 4254
galaxy = galdic[5]
galveldic = {'NGC4254': 2388 , 'NGC4535': 1954  , 'NGC3351': 775, 'NGC2835': 867, 'NGC0628':651}
galvel = galveldic[galaxy]
galebv = {'NGC4254': 0.0334 , 'NGC4535': 0.0168 , 'NGC3351': 0.0239, 'NGC2835': 0.0859, 'NGC0628': 0.0607}
ebv = galebv[galaxy]
infile = open(f"/home/habjan/jupfiles/data/{galaxy}_refit+SITELLEfits_data.fits",'rb')
data = Table.read(infile)

hdul = fits.open(f'/home/habjan/jupfiles/data/{galaxy}_SITELLE_Spectra.fits')
sitlam = hdul[0].data
sitspec = hdul[1].data - hdul[2].data

infile = f"/home/habjan/jupfiles/data/{galaxy}_cube.hdf5"
cube = SpectralCube(infile)

galaxy

### Find the highest SNR [OII] regioins

In [ ]:
brightreg = np.where(data['OII3727_FLUX_CORR'] > 20*data['OII3727_FLUX_CORR_ERR'])[0]
brightreg

# Electron Density and Temperature 

### Import PyNeb diagnostics

In [ ]:
diags = pn.Diagnostics(addAll=True)

### Create [OII] temperture diagnostic

In [ ]:
diags.addDiag('[OII] b3727/b7325', ('O2', '(L(3726)+L(3729))/(L(7319)+L(7320)+L(7330)+L(7331))', 
                                    'RMS([E(3727A+),E(7319A+)*L(7319A+)/(L(7319A+)+L(7330A+)),E(7330A+)*L(7330A+)/(L(7319A+)+L(7330A+))])'))

### Test an [OII] density calcualtion

In [ ]:
test = data[brightreg[3]]

te, ne = diags.getCrossTemDen(diag_tem = '[OII] b3727/b7325', diag_den = '[SII] 6731/6716', 
                                    value_tem = (test['OII3727_FLUX_CORR'])/(test['OII7319_FLUX_CORR_REFIT']+test['OII7330_FLUX_CORR_REFIT']), 
                                    value_den = test['SII6730_FLUX_CORR']/test['SII6716_FLUX_CORR'], 
                                    guess_tem = 10**4)
te, ne

### hypothetical flux [OII] flux ratios and find the resultant Density and Temperature

In [ ]:
numiters = 100
lowlim = 1.2
uplim = 1.5
fluxratios = np.linspace(lowlim, uplim, numiters)
teoii = np.zeros(numiters)
neoii = np.zeros(numiters)
tesii = np.zeros(numiters)
nesii = np.zeros(numiters)

for i in range(numiters):
    teoii[i], neoii[i] = diags.getCrossTemDen(diag_tem = '[OII] b3727/b7325', diag_den = '[SII] 6731/6716', 
                                              value_tem = 150, 
                                              value_den = 1/fluxratios[i], 
                                              guess_tem = 10**4)

### Find what density value the flux ratio starts changing very little

In [ ]:
lowden = 0
for i in range(1, len(neoii)): 
    if (neoii[i-1] - neoii[i])/neoii[i] > 0.05:
        lowden = i
    if lowden != 0:
        break
lowden

### Plot the Density results

In [ ]:
plt.scatter(neoii, fluxratios, s=5, color='blue', label = '[SII] Density')

plt.axvline(50, c = 'red', label=r'50 cm$^{-3}$')
plt.axvline(100, c = 'green', label=r'100 cm$^{-3}$')
plt.axvline(neoii[lowden], c = 'k', label=f'{round(neoii[lowden], 1)}' r'cm$^{-3}$')

plt.ylabel('Flux Ratio')
plt.xlabel(r'Electron Density [cm$^{-3}$]')
plt.xlim(5, None)
plt.xscale('log')
plt.legend(fontsize=8)

### Plot the Temperature Results

In [ ]:
plt.scatter(fluxratios, teoii, s=10, marker='.', color='blue', label = '[OII] 3729/3726')
plt.scatter(fluxratios, tesii, s=10, marker='.', color='red', label = '[SII] 6716/6731')

plt.xlabel('Flux Ratio')
plt.ylabel(r'Electron Temperature [K]')
plt.ylim(0, None)
#plt.yscale('log')

# [SII] and [OII] data comparisons

### Calculate the [SII] densities/temeperatures and flux ratios

In [ ]:
nebrat = np.zeros(len(data))
nebte = np.zeros(len(data))
nebne = np.zeros(len(data))


for i in range(len(data)):
    try:
        nebrat[i] = data[i]['SII6716_FLUX_CORR'] / data[i]['SII6730_FLUX_CORR']
    except: 
        nebrat[i] = np.nan
    
    try: 
        nebte[i], nebne[i] = diags.getCrossTemDen(diag_tem = '[SII] 6731/6716', diag_den = '[SII] 6731/6716', 
                                                value_tem = 1/nebrat[i], 
                                                value_den = 1/nebrat[i], 
                                                guess_tem = 10**4)
    except: 
        nebte[i], nebne[i] = np.nan, np.nan

### Write a function that finds the [OII] fluxes with a different input amplitude

In [ ]:
def oii_amp(phdata, inspec, inlam, incube, ingalvel, guess_amp):

    OII3726_flux, OII3729_flux = np.zeros(len(phdata)), np.zeros(len(phdata))
    OIIvelocity = np.zeros(len(phdata))

    broadening_gvar = gvar.gvar(1, 20)

    S2 = pn.Atom('S', 2)
    O2 = pn.Atom('O', 2)
    temp = 10**4
    fitshape = 'sinc'
    OII3726 = Lines().get_line_cm1('[OII]3726')
    OII3729 = Lines().get_line_cm1('[OII]3729')
    for i in range(len(phdata)):
        
        inspectrum = inspec[i]
        inspectrum[np.isnan(inspectrum)] = 0

        wave3726 = 3726.032
        red3726 = 1/((wave3726*(data[i]['HA6562_VEL']+ingalvel)/(299792) + wave3726) * 10**-8)
        wave3729 = 3728.815
        red3729 = 1/((wave3729*(data[i]['HA6562_VEL']+ingalvel)/(299792) + wave3729) * 10**-8)

        n1 = [np.where(inlam > incube.params.filter_range[0])[0][0],np.where(inlam > red3729 - 50)[0][0]]
        n2 = [np.where(inlam > red3729 + 50)[0][0], np.where(inlam > incube.params.filter_range[1])[0][0]]
        noise = (np.mean(inspectrum[n1[0]:n1[1]]) + np.mean(inspectrum[n2[0]:n2[1]])) / 2
        noisestd = (np.std(inspectrum[n1[0]:n1[1]]) + np.std(inspectrum[n2[0]:n2[1]])) / 2
        inspectrum = (inspectrum) - (noise)

        invel = data[i]['HA6562_VEL']+ingalvel
        velocity_gvar = gvar.gvar(invel, 20)    

        amp_ratio = amp_ratio_from_flux_ratio(OII3729,OII3726,guess_amp)

        try:

            fitvel = fit_lines_in_spectrum(inspectrum, [OII3726, OII3729], 
                        step=incube.params.step, order=incube.params.order, nm_laser=incube.params.nm_laser, theta=corr2theta(incube.params.axis_corr), zpd_index=incube.params.zpd_index, 
                        wavenumber=True,
                        apodization=1, 
                        fmodel=fitshape,
                        pos_def=['1', '1'],
                        pos_cov=[velocity_gvar],
                        amp_def=['1', '1'], 
                        amp_guess=[1, amp_ratio],
                        )

            wave3726 = 3726.032
            red3726 = 1/((wave3726*(fitvel['velocity'][0])/(299792) + wave3726) * 10**-8)
            wave3729 = 3728.815
            red3729 = 1/((wave3729*(fitvel['velocity'][1])/(299792) + wave3729) * 10**-8)
            velocity_gvar = gvar.gvar(0, 1) 

            fit = fit_lines_in_spectrum(inspectrum, [red3726, red3729], 
                      step=incube.params.step, order=incube.params.order, nm_laser=incube.params.nm_laser, theta=corr2theta(incube.params.axis_corr), zpd_index=incube.params.zpd_index, 
                      wavenumber=True,
                      apodization=1, 
                      fmodel=fitshape,
                      pos_def='fixed',
                      amp_def='free',
                      )
            
            OII3726_flux[i], OII3729_flux[i] = fit['flux'] 
            OIIvelocity[i] = fitvel['velocity'][0] - ingalvel

        except: 
            
            OII3726_flux[i], OII3729_flux[i] = np.nan, np.nan
            OIIvelocity[i] = np.nan
    
    return OII3726_flux, OII3729_flux, OIIvelocity

### Obtain [OII] fluxes for a given amplitude ratio

In [ ]:
inamp_ratio = 1.4
oii3726, oii3729, oii_velocity = oii_amp(data, sitspec, sitlam, cube, galvel, inamp_ratio)

### Calcualte densities, temperatures and line ratios for [OII] 

In [ ]:
oiirat = np.zeros(len(data))
oiite = np.zeros(len(data))
oiine = np.zeros(len(data))


for i in range(len(data)):
    try:
        oiirat[i] = oii3729[i] / oii3726[i]
    except: 
        oiirat[i] = np.nan
    
    try: 
        oiite[i], oiine[i] = diags.getCrossTemDen(diag_tem = '[OII] 3726/3729', diag_den = '[OII] 3726/3729', 
                                                value_tem = 1/oiirat[i], 
                                                value_den = 1/oiirat[i], 
                                                guess_tem = 10**4)
    except: 
        oiite[i], oiine[i] = np.nan, np.nan

### Plot histograms of the [OII] and [SII] flux ratios

In [ ]:
inoiirat = oiirat[~np.isnan(oiirat)]
innebrat = nebrat[~np.isnan(nebrat)]

up = 3
inoiirat = inoiirat[inoiirat < up]
innebrat = innebrat[innebrat < up]

low = 0
inoiirat = inoiirat[low < inoiirat]
innebrat = innebrat[low < innebrat]

oiimed = np.median(inoiirat)
nebmed = np.median(innebrat)

a=np.random.random(100)*0.5 #a uniform distribution
b=1-np.random.normal(size=100)*0.1 #a normal distribution 
bins=np.histogram(np.hstack((inoiirat,innebrat)), bins=100)[1]; #get the bin edges

plt.hist(inoiirat, color='k', edgecolor='k', alpha = 1, bins=bins, label = f'[OII] 3729/3726: {len(inoiirat)}');
plt.hist(innebrat, color='green', edgecolor='k', alpha = 0.6, bins=bins, label = f'[SII] 6716/6731: {len(innebrat)}');
plt.axvline(oiimed, color='k', linestyle='--', label = f'Median: {round(oiimed, 2)}')
plt.axvline(nebmed, color='green', alpha=0.5, linestyle='--', label = f'Median: {round(nebmed, 2)}')
plt.title(f'[OII] fitted with {inamp_ratio} Amplitude guess')
plt.ylabel('count')
plt.xlabel('Flux Ratio')
plt.legend()

### Plot the [OII] and [SII] densities

In [ ]:
inoiirat = oiine[~np.isnan(oiine)]
innebrat = nebne[~np.isnan(nebne)]

up = 1000
inoiirat = inoiirat[inoiirat < up]
innebrat = innebrat[innebrat < up]

low = 0
inoiirat = inoiirat[low < inoiirat]
innebrat = innebrat[low < innebrat]

oiimed = np.median(inoiirat)
nebmed = np.median(innebrat)

a=np.random.random(100)*0.5 #a uniform distribution
b=1-np.random.normal(size=100)*0.1 #a normal distribution 
bins=np.histogram(np.hstack((inoiirat,innebrat)), bins=100)[1]; #get the bin edges

plt.hist(inoiirat, color='k', edgecolor='k', alpha = 1, bins=bins, label = f'[OII] Density: {len(inoiirat)}');
plt.hist(innebrat, color='green', edgecolor='k', alpha = 0.6, bins=bins, label = f'[SII] Density: {len(innebrat)}');
plt.axvline(oiimed, color='k', linestyle='--', label = f'Median: {round(oiimed, 2)}')
plt.axvline(nebmed, color='green', alpha=0.5, linestyle='--', label = f'Median: {round(nebmed, 2)}')
plt.title(f'[OII] fitted with {inamp_ratio} Amplitude guess')
plt.ylabel('count')
plt.xlabel(r'Electron Density [cm$^{-3}$]')
plt.legend()

### Compare the [OII] velocities to the velocities in the Nebular catalog

In [ ]:
havels = np.array([data[i]['HA6562_VEL'] for i in range(len(data))])
niivels = np.array([data[i]['NII6583_VEL'] for i in range(len(data))]) 
oiiivels = np.array([data[i]['OIII5006_VEL'] for i in range(len(data))]) 

hamed = np.median(havels)
niimed = np.median(niivels)
oiiimed = np.median(oiiivels)
oiimed = np.median(oii_velocity)

havels = havels[~np.isnan(havels)]
niivels = niivels[~np.isnan(niivels)]
oiiivels = oiiivels[~np.isnan(oiiivels)]
oii_velocity1 = oii_velocity[~np.isnan(oii_velocity)]

a=np.random.random(100)*0.5 #a uniform distribution
b=1-np.random.normal(size=100)*0.1 #a normal distribution 
bins=np.histogram(np.hstack((havels, niivels, oiiivels, oii_velocity)), bins=50)[1]; #get the bin edges

plt.hist(havels, color='k', edgecolor='k', alpha = 0.6, bins=bins, label = r'[H$\alpha$] Velocity:'f'{len(havels)}');
plt.hist(niivels, color='green', edgecolor='k', alpha = 0.6, bins=bins, label = f'[NII] Velocity: {len(niivels)}');
plt.hist(oiiivels, color='red', edgecolor='k', alpha = 0.6, bins=bins, label = f'[OIII] Velocity: {len(oiiivels)}');
plt.hist(oii_velocity1, color='blue', edgecolor='k', alpha = 0.6, bins=bins, label = f'[OII] Velocity: {len(oii_velocity1)}');

plt.axvline(hamed, color='k', linestyle='--', label = f'Median: {round(hamed, 2)} 'r'[$\frac{km}{s}$]')
plt.axvline(niimed, color='green', alpha=0.5, linestyle='--', label = f'Median: {round(niimed, 2)} 'r'[$\frac{km}{s}$]')
plt.axvline(oiiimed, color='red', alpha=0.5, linestyle='--', label = f'Median: {round(oiiimed, 2)} 'r'[$\frac{km}{s}$]')
plt.axvline(oiimed, color='blue', alpha=0.5, linestyle='--', label = f'Median: {round(oiimed, 2)} 'r'[$\frac{km}{s}$]')

plt.title(f'[OII] fitted with {inamp_ratio} Amplitude guess')
plt.ylabel('count')
plt.xlabel(r'Rotational Velocity [$\frac{km}{s}$]')
plt.legend()

# Extract [OII]3727 (single feature) fluxes

### The function to extract fluxes

In [ ]:
def oii_3727(phdata, inspec, inwave, incube, ingalvel, inebv, insnval):

    out_fluxes = np.zeros(len(phdata))
    out_velocities = np.zeros(len(phdata))

    broadening_gvar = gvar.gvar(1, 20)

    S2 = pn.Atom('S', 2)
    O2 = pn.Atom('O', 2)
    temp = 10**4
    fitshape = 'sinc'
    OII3726 = Lines().get_line_cm1('[OII]3726')
    OII3729 = Lines().get_line_cm1('[OII]3729')
    for i in range(len(phdata)):
        
        inspectrum = inspec[i]
        inspectrum[np.isnan(inspectrum)] = 0

        wave3729 = 3728.815
        c = 299792
        red3729 = 1/((wave3729*(phdata[i]['HA6562_VEL']+galvel)/(c) + wave3729) * 10**-8)

        n1 = [np.where(inwave > incube.params.filter_range[0])[0][0],np.where(inwave > red3729 - 175)[0][0]]
        n2 = [np.where(inwave > red3729 + 175)[0][0], np.where(inwave > incube.params.filter_range[1])[0][0]]
        noise = (np.mean(inspectrum[n1[0]:n1[1]]) + np.mean(inspectrum[n2[0]:n2[1]])) / 2
        noisestd = (np.std(inspectrum[n1[0]:n1[1]]) + np.std(inspectrum[n2[0]:n2[1]])) / 2
        inspectrum = (inspectrum) - (noise)

        invel = phdata[i]['HA6562_VEL']+ingalvel   #Not being used rn because the mean of gvar is 0 and the input wave is already accounting for redshift
        velocity_gvar = gvar.gvar(invel, 3)
        
        try:
            fit = fit_lines_in_spectrum(inspectrum, [OII3729], 
                      step=incube.params.step, order=incube.params.order, nm_laser=incube.params.nm_laser, theta=corr2theta(incube.params.axis_corr), zpd_index=incube.params.zpd_index, 
                      wavenumber=True,
                      apodization=1, 
                      fmodel=fitshape,
                      pos_def=['1'],
                      pos_cov=[velocity_gvar],
                      amp_def=['1'], 
                      amp_guess=[1],
                      )
            
            snr = fit['lines_params'][0][1] / noisestd

            if snr > insnval:
                flux = float(fit['flux'][0])
                redline = 1/(float(fit['lines_params'][0][2]) * 10**-8)

                R_V = 3.1
                mwcorr = remove(extinction.odonnell94(np.array([redline]), inebv*R_V, R_V), flux)

                A_V = phdata[i]['EBV'] * R_V
                corrflux = remove(extinction.odonnell94(np.array([redline]), A_V, R_V), mwcorr)

                out_fluxes[i] = corrflux[0]
                out_velocities[i] = (((redline - wave3729)/wave3729)*c) - ingalvel
            else:
                out_fluxes[i] = np.nan
                out_velocities[i] = np.nan
        except:
            out_fluxes[i] = np.nan
            out_velocities[i] = np.nan
            continue
    
    return out_fluxes, out_velocities

### Run the code

In [ ]:
oii3727, oii3727_velocity = oii_3727(data, sitspec, sitlam, cube, galvel, ebv, 3);

print(f'There are {len(np.where(~np.isnan(oii3727))[0])} [OII]3727 flux measurements')

### Compare [OII]3727 velocities

In [ ]:
havels = np.array([data[i]['HA6562_VEL'] for i in range(len(data))])
niivels = np.array([data[i]['NII6583_VEL'] for i in range(len(data))]) 
oiiivels = np.array([data[i]['OIII5006_VEL'] for i in range(len(data))]) 

havels = havels[~np.isnan(havels)]
niivels = niivels[~np.isnan(niivels)]
oiiivels = oiiivels[~np.isnan(oiiivels)]
oii_velocity1 = oii3727_velocity[~np.isnan(oii3727_velocity)]

hamed = np.median(havels)
niimed = np.median(niivels)
oiiimed = np.median(oiiivels)
oiimed = np.median(oii_velocity1)

a=np.random.random(100)*0.5 #a uniform distribution
b=1-np.random.normal(size=100)*0.1 #a normal distribution 
bins=np.histogram(np.hstack((havels, niivels, oiiivels, oii_velocity1)), bins=50)[1]; #get the bin edges

plt.hist(havels, color='k', edgecolor='k', alpha = 0.6, bins=bins, label = r'[H$\alpha$] Velocity:'f'{len(havels)}');
plt.hist(niivels, color='green', edgecolor='k', alpha = 0.6, bins=bins, label = f'[NII] Velocity: {len(niivels)}');
plt.hist(oiiivels, color='red', edgecolor='k', alpha = 0.6, bins=bins, label = f'[OIII] Velocity: {len(oiiivels)}');
plt.hist(oii_velocity1, color='blue', edgecolor='k', alpha = 0.6, bins=bins, label = f'[OII] Velocity: {len(oii_velocity1)}');

plt.axvline(hamed, color='k', linestyle='--', label = f'Median: {round(hamed, 2)} 'r'[$\frac{km}{s}$]')
plt.axvline(niimed, color='green', alpha=0.5, linestyle='--', label = f'Median: {round(niimed, 2)} 'r'[$\frac{km}{s}$]')
plt.axvline(oiiimed, color='red', alpha=0.5, linestyle='--', label = f'Median: {round(oiiimed, 2)} 'r'[$\frac{km}{s}$]')
plt.axvline(oiimed, color='blue', alpha=0.5, linestyle='--', label = f'Median: {round(oiimed, 2)} 'r'[$\frac{km}{s}$]')

#plt.title(f'[OII] fitted with {inamp_ratio} Amplitude guess')
plt.ylabel('count')
plt.xlabel(r'Rotational Velocity [$\frac{km}{s}$]')
plt.legend()

### Find the Temperature and Densities using getTemDen

### Import atom class information

In [ ]:
O1 = pn.Atom('O', 1)
O2 = pn.Atom('O', 2)
O3 = pn.Atom('O', 3)
S2 = pn.Atom('S', 2)
S3 = pn.Atom('S', 3)

### Find [SII] Density for a region

In [ ]:
reg = 73
temp = 10**4

S2ratio = data[reg]['SII6730_FLUX_CORR'] / data[reg]['SII6716_FLUX_CORR']
density = S2.getTemDen(int_ratio=S2ratio, tem=temp, to_eval= 'L(6731)/L(6717)')   #wave1=6731, wave2=6717)
density

### Find [OII] temperature for a region

In [ ]:
O2ratio = oii3727[reg] / (data[reg]['OII7319_FLUX_CORR_REFIT']+data[reg]['OII7330_FLUX_CORR_REFIT'])
temp = O2.getTemDen(int_ratio=O2ratio, den=density, to_eval='(L(3726)+L(3729))/(L(7319)+L(7320)+L(7330)+L(7331))')
temp

### Find [SIII] temperatue for a region

In [ ]:
S3ratio = data[reg]['SIII6312_FLUX_CORR_REFIT']/data[reg]['SIII9068_FLUX_CORR']
temp_s3 = S3.getTemDen(int_ratio=S3ratio, den=density, to_eval='L(6312)/L(9068)')
temp_s3

### Compare to getCrossTemDen

In [ ]:
density/ne, temp/te

### Wrtie code to find the temperature and density in every region using getTemDen

In [ ]:
oiite = np.zeros(len(data))
oiine = np.zeros(len(data))

tesiii = np.zeros(len(data))
teoiii = np.zeros(len(data))
nesiii = np.zeros(len(data))


for i in range(len(data)):
    try: 
        S2ratio = data[i]['SII6730_FLUX_CORR'] / data[i]['SII6716_FLUX_CORR']
        oiine[i] = S2.getTemDen(int_ratio=S2ratio, tem=temp, to_eval= 'L(6731)/L(6717)')
        O2ratio = oii3727[i] / (data[i]['OII7319_FLUX_CORR_REFIT']+data[i]['OII7330_FLUX_CORR_REFIT'])
        oiite[i] = O2.getTemDen(int_ratio=O2ratio, den=density, to_eval='(L(3726) + L(3729)) / (L(7319) + L(7320) + L(7330) + L(7331))')
    except: 
        oiite[i], oiine[i] = np.nan, np.nan
    
    try: 
        nesiii[i] = oiine[i]
        S3ratio = data[i]['SIII6312_FLUX_CORR_REFIT']/data[i]['SIII9068_FLUX_CORR']
        tesiii[i] = S3.getTemDen(int_ratio=S3ratio, den=density, to_eval='L(6312)/L(9068)') 

        if tesiii[i] < 14000:
            teoiii[i] = 0.7092 * tesiii[i] + 3609.9
        else: 
            teoiii[i] = np.nan
    except: 
        tesiii[i], nesiii[i] = np.nan, np.nan

# Metallicity

### Import atom class information and ICF

In [ ]:
icf = pn.ICF()
O1 = pn.Atom('O', 1)
O2 = pn.Atom('O', 2)
O3 = pn.Atom('O', 3)
icf_list = ['Ial06_16']

### Find the [SII] densities, [OII] temperatures, [SII] densities (with [SIII]), and [OIII] temperatures

In [ ]:
oiite_cross = np.zeros(len(data))
oiine_cross = np.zeros(len(data))

tesiii_cross = np.zeros(len(data))
teoiii_cross = np.zeros(len(data))
nesiii_cross = np.zeros(len(data))


for i in range(len(data)):
    
    try: 
        oiite_cross[i], oiine_cross[i] = diags.getCrossTemDen(diag_tem = '[OII] b3727/b7325', diag_den = '[SII] 6731/6716', 
                                                value_tem = (oii3727[i])/(data[i]['OII7319_FLUX_CORR_REFIT']+data[i]['OII7330_FLUX_CORR_REFIT']), 
                                                value_den = data[i]['SII6730_FLUX_CORR']/data[i]['SII6716_FLUX_CORR'], 
                                                guess_tem = 10**4)
    except: 
        oiite_cross[i], oiine_cross[i] = np.nan, np.nan
    
    try: 
        tesiii_cross[i], nesiii_cross[i] = diags.getCrossTemDen(diag_tem = '[SIII] 6312/9069', diag_den = '[SII] 6731/6716', 
                                                    value_tem = data[i]['SIII6312_FLUX_CORR_REFIT']/data[i]['SIII9068_FLUX_CORR'], 
                                                    value_den = data[i]['SII6730_FLUX_CORR']/data[i]['SII6716_FLUX_CORR'], 
                                                    guess_tem = 10**4)

        if tesiii_cross[i] < 14000:
            teoiii_cross[i] = 0.7092 * tesiii_cross[i] + 3609.9
        else: 
            teoiii_cross[i] = np.nan
    except: 
        tesiii_cross[i], nesiii_cross[i] = np.nan, np.nan

### Plot [OII] Temperatures and [SII] Densities

In [ ]:
x = oiine
y = oiite

fig = plt.figure(figsize=(5, 5))

gs = fig.add_gridspec(2, 2,  width_ratios=(4, 1), height_ratios=(1, 4),
                      left=0.1, right=0.9, bottom=0.1, top=0.9,
                      wspace=0.0, hspace=0.0)

ax = fig.add_subplot(gs[1, 0])
ax_histx = fig.add_subplot(gs[0, 0], sharex=ax)
ax_histy = fig.add_subplot(gs[1, 1], sharey=ax)

# Cmap info
oii_where = np.where(~np.isnan(y))[0]

snr_7319 = np.array([data[i]['OII7319_FLUX_CORR_REFIT'][0]/data[i]['OII7319_FLUX_CORR_REFIT_ERR'][0] for i in range(len(data))])
snr_7330 = np.array([data[i]['OII7330_FLUX_CORR_REFIT'][0]/data[i]['OII7330_FLUX_CORR_REFIT_ERR'][0] for i in range(len(data))])

snr_7319 = snr_7319[oii_where]
snr_7330 = snr_7330[oii_where]

# the scatter plot
cax = ax.scatter(x[oii_where], y[oii_where], s=100, marker='.', c=snr_7319, cmap='bwr', edgecolors='k')
ax.set_xlim([0, None])
ax.set_ylim([None, None])
#fig.suptitle('NII derived Temperature versus SII derived Density')
ax.set_xlabel('[SII] Density [$cm^{-3}$]')
ax.set_ylabel('[OII] Temperature [K]')
#ax.set_yscale('log')

# the histograms
ax_histx.hist(x, bins=60, color='blue');
ax_histy.hist(y, bins=15,color='blue', orientation='horizontal');
ax_histx.tick_params(axis="x", labelbottom=False)
ax_histy.tick_params(axis="y", labelleft=False)
ax.tick_params(axis="y", labelright=False)

cbar = fig.colorbar(cax, label='[OII]7320 SNR')
plt.show()

### Test an [OII] metallicity calculation

In [ ]:
reg = 73

OII = O2.getIonAbundance(int_ratio = oii3727[reg]+data[reg]['OII7319_FLUX_CORR_REFIT']+data[reg]['OII7330_FLUX_CORR_REFIT'], tem = oiite[reg], den= oiine[reg], 
                         to_eval='L(3726)+L(3729)+L(7319)+L(7320)+L(7330)+L(7331)', Hbeta = data[reg]['HB4861_FLUX_CORR'])
OIII = O3.getIonAbundance(int_ratio = data[reg]['OIII5006_FLUX_CORR'], tem = teoiii[reg], den= nesiii[reg], 
                          to_eval='L(5007)', Hbeta = data[reg]['HB4861_FLUX_CORR'])
            
abunlist = {'O2': OII, 'O3': OIII}
abundance = icf.getElemAbundance(abunlist, icf_list)
met = 12 + np.log10(abundance['Ial06_16'])
met[0]

### Use the [SII] temperatures/densities and the [OII]3727 line to calculate metallicity

In [ ]:
mets = np.zeros(len(data))
den_cutoff = 50

for i in range(len(data)):
    if oiine[i] > den_cutoff:
        OII = O2.getIonAbundance(int_ratio = oii3727[i]+data[i]['OII7319_FLUX_CORR_REFIT']+data[i]['OII7330_FLUX_CORR_REFIT'],
                            tem = oiite[i], den= oiine[i], 
                            to_eval='L(3726)+L(3729)+L(7319)+L(7320)+L(7330)+L(7331)', Hbeta = data[i]['HB4861_FLUX_CORR'])

        OIII = O3.getIonAbundance(int_ratio = data[i]['OIII5006_FLUX_CORR'],
                              tem = teoiii[i], den= nesiii[i], 
                              to_eval='L(5007)', Hbeta = data[i]['HB4861_FLUX_CORR'])
    else:
        OII, OIII = np.nan, np.nan
            
    abunlist = {'O2': OII, 'O3': OIII}
    abundance = icf.getElemAbundance(abunlist, icf_list)
    mets[i] = 12 + np.log10(abundance['Ial06_16'])

### Create a list of Radii

In [ ]:
distdic = {'NGC4254': 13.1 , 'NGC4535': 16.60  , 'NGC3351': 10.0, 'NGC2835': 10.4, 'NGC0628':7.5}
dist = distdic[galaxy]
kpc = data['deproj_dist'] / 206265 * (dist * 1000)
nkpc = np.array([kpc[i] for i in range(len(kpc))])

### Make lists and polynomial fits of data to be plotted

In [ ]:
s = np.array([data[i]['met_scal'] for i in range(len(data)) if np.isnan(data[i]['met_scal']) == False])
scals = np.array([data[i]['met_scal'] for i in range(len(data))])
scal = np.polyfit(nkpc[np.where(np.isnan(data['met_scal']) == False)], s, 1)
scalfunc = np.poly1d(scal)

dirc = np.array([mets[i] for i in range(len(data)) if np.isnan(mets[i]) == False])
direct = np.polyfit(nkpc[np.where(~np.isnan(mets))[0]], dirc, 1)
dirfunc = np.poly1d(direct)

seq = np.linspace(-3, 15, num = 20)

### Plot the Metallicity results

In [ ]:
fig = plt.figure()
fig.set_size_inches(8, 4)

plt.scatter(nkpc, scals, s=5, marker='.', color='orange', label = f'Scal-PG16 Nebular catalog: {len(s)}')
plt.plot(seq, scalfunc(seq), c='orange',  linestyle="--", label=f'Scal-PG16: y = {round(scalfunc[1],3)}x + {round(scalfunc[0],3)}')

plt.scatter(nkpc, mets, s=25, marker='.', color='blue', label = f'Direct: {len(dirc)}')
plt.plot(seq, dirfunc(seq), c='blue', linestyle="--", label=f'Direct: y = {round(dirfunc[1],3)}x + {round(dirfunc[0],3)}')

plt.xlabel('Galactocentric Radius [kpc]')
plt.ylabel(r'Metallicity [12 + log(O/H)]')
#plt.ylim(8, 8.5)
plt.xlim(0, np.max(nkpc)*1.05)

plt.title(f'Metallicity calculations in {galaxy}')
plt.legend(bbox_to_anchor=(1.43, 0.7), fontsize="9",loc='center right');
plt.figtext(1.0905, 0.3, f"Direct STD: {round(np.std(dirc), 3)} \n Scal-PG16 Nebular catalog STD: {round(np.std(s),3)}", ha="center", bbox={"facecolor":"white", "alpha":0.5, "pad":5})

### Plot only the [NII] derived metallicities with a color scale for the [OII]3727 fluxes

In [ ]:
fig = plt.figure()
fig.set_size_inches(8, 4)

color_num = 4
oii_where = np.where(~np.isnan(mets))[0]

hbeta = np.array([data[i]['HB4861_FLUX_CORR'] for i in range(len(data))])
snr_7319 = np.array([data[i]['OII7319_FLUX_CORR_REFIT'][0]/data[i]['OII7319_FLUX_CORR_REFIT_ERR'][0] for i in range(len(data))])
snr_7330 = np.array([data[i]['OII7330_FLUX_CORR_REFIT'][0]/data[i]['OII7330_FLUX_CORR_REFIT_ERR'][0] for i in range(len(data))])
density = np.array([oiine[i] for i in range(len(data))])

hbeta_vals = np.log10(hbeta[oii_where])
snr_7319 = snr_7319[oii_where]
snr_7330 = snr_7330[oii_where]
density = density[oii_where]
invals_dic = {1:hbeta_vals, 2:snr_7319, 3:snr_7330, 4:density}

plt.scatter(nkpc, scals, s=5, marker='.', color='orange', alpha=0.5, label = f'Scal-PG16 Nebular catalog: {len(s)}')
plt.plot(seq, scalfunc(seq), c='orange',  linestyle="--", label=f'Scal-PG16: y = {round(scalfunc[1],3)}x + {round(scalfunc[0],3)}')

plt.scatter(nkpc[oii_where], mets[oii_where], s=100, marker='.', c=invals_dic[color_num], cmap='bwr', edgecolors='k', label = f'Direct: {len(oii_where)}')
plt.plot(seq, dirfunc(seq), c='k', linestyle="--", label=f'Direct: y = {round(dirfunc[1],3)}x + {round(dirfunc[0],3)}')

plt.xlabel('Galactocentric Radius [kpc]')
plt.ylabel(r'Metallicity [12 + log(O/H)]')
plt.ylim(8.25, 9.4)
plt.xlim(0, np.max(nkpc)*1.05)

plt.title(f'Metallicity calculations in {galaxy}')
plt.legend(bbox_to_anchor=(1.8, 0.65), fontsize="9",loc='center right');
plt.figtext(1.085, 0.325, f"Direct STD: {round(np.std(dirc), 3)} \n Scal-PG16 Nebular catalog STD: {round(np.std(s),3)}", ha="center", bbox={"facecolor":"white", "alpha":0.5, "pad":5})

label_dict = {1:r'H$\beta$ [log($\frac{erg}{cm^{2} \cdot s}$)]', 2:'SNR of [OII]7320', 3:'SNR of [OII]7330', 4:r'[SII] Density [$cm^{-3}$]'}
label_in = label_dict[color_num]
plt.colorbar(label=label_in)

### Comparing [OII]3727 with [OII]3726 + 3729

In [ ]:
oii_combined = np.array([data[i]['OII3726_FLUX_CORR'][0] + data[i]['OII3729_FLUX_CORR'][0] for i in range(len(data))])
plot_list = np.array([i for i in range(len(data)) if ~np.isnan(oii_combined[i]) and ~np.isnan(oii3727[i])])

plt.scatter(oii3727, oii_combined, marker='.', s=50, c='k', label=f'count: {len(plot_list)}')

plt.plot(np.linspace(1, 10**11, 10), np.linspace(1, 10**11, 10), c='k', label='1-to-1 line')

plt.title(f'[OII]3726,3729 versus [OII]3727 Flux comparison in {galaxy}')
plt.ylabel('[OII]3726 + [OII]3729')
plt.xlabel('[OII]3729')
lowb = 10**4
upb = 10**8.6
plt.xlim(lowb, upb)
plt.ylim(lowb, upb)
plt.xscale('log')
plt.yscale('log')
plt.legend()

# Comparing getCrossTemDen and getTemDen results

### Plot [OII] temperatues

In [ ]:
x = oiite
y = oiite_cross

fig = plt.figure(figsize=(5, 5))

gs = fig.add_gridspec(2, 2,  width_ratios=(4, 1), height_ratios=(1, 4),
                      left=0.1, right=0.9, bottom=0.1, top=0.9,
                      wspace=0.0, hspace=0.0)

ax = fig.add_subplot(gs[1, 0])
ax_histx = fig.add_subplot(gs[0, 0], sharex=ax)
ax_histy = fig.add_subplot(gs[1, 1], sharey=ax)

# Cmap info
oii_where = np.where(~np.isnan(y))[0]

snr_7319 = np.array([data[i]['OII7319_FLUX_CORR_REFIT'][0]/data[i]['OII7319_FLUX_CORR_REFIT_ERR'][0] for i in range(len(data))])
snr_7330 = np.array([data[i]['OII7330_FLUX_CORR_REFIT'][0]/data[i]['OII7330_FLUX_CORR_REFIT_ERR'][0] for i in range(len(data))])

snr_7319 = snr_7319[oii_where]
snr_7330 = snr_7330[oii_where]

# the scatter plot
cax = ax.scatter(x[oii_where], y[oii_where], s=100, marker='.', c=snr_7319, cmap='bwr', edgecolors='k')
ax.set_xlim([5000, 10000])
ax.set_ylim([5000, 10000])
ax.plot(np.linspace(0, 10**6, 10), np.linspace(0, 10**6, 10), color='k')
#fig.suptitle('NII derived Temperature versus SII derived Density')
ax.set_xlabel('[OII] Temperature [K] from getTemDen')
ax.set_ylabel('[OII] Temperature [K] from getCrossTemDen')
#ax.set_yscale('log')

# the histograms
ax_histx.hist(x, bins=60, color='blue');
ax_histy.hist(y, bins=15,color='blue', orientation='horizontal');
ax_histx.tick_params(axis="x", labelbottom=False)
ax_histy.tick_params(axis="y", labelleft=False)
ax.tick_params(axis="y", labelright=False)

cbar = fig.colorbar(cax, label='[OII]7320 SNR')
plt.show()

### Plot [SII] Densities

In [ ]:
x = oiine
y = oiine_cross

fig = plt.figure(figsize=(5, 5))

gs = fig.add_gridspec(2, 2,  width_ratios=(4, 1), height_ratios=(1, 4),
                      left=0.1, right=0.9, bottom=0.1, top=0.9,
                      wspace=0.0, hspace=0.0)

ax = fig.add_subplot(gs[1, 0])
ax_histx = fig.add_subplot(gs[0, 0], sharex=ax)
ax_histy = fig.add_subplot(gs[1, 1], sharey=ax)

# Cmap info
oii_where = np.where(~np.isnan(y))[0]

snr_7319 = np.array([data[i]['OII7319_FLUX_CORR_REFIT'][0]/data[i]['OII7319_FLUX_CORR_REFIT_ERR'][0] for i in range(len(data))])
snr_7330 = np.array([data[i]['OII7330_FLUX_CORR_REFIT'][0]/data[i]['OII7330_FLUX_CORR_REFIT_ERR'][0] for i in range(len(data))])

snr_7319 = snr_7319[oii_where]
snr_7330 = snr_7330[oii_where]

# the scatter plot
cax = ax.scatter(x[oii_where], y[oii_where], s=100, marker='.', c=snr_7319, cmap='bwr', edgecolors='k')
ax.set_xlim([0, None])
ax.set_ylim([0, None])
ax.plot(np.linspace(0, 10**6, 10), np.linspace(0, 10**6, 10), color='k')
#fig.suptitle('NII derived Temperature versus SII derived Density')
ax.set_xlabel(r'[SII] Density [cm$^{-3}$] from getTemDen')
ax.set_ylabel(r'[SII] Density [cm$^{-3}$] from getCrossTemDen')
#ax.set_yscale('log')

# the histograms
ax_histx.hist(x, bins=60, color='blue');
ax_histy.hist(y, bins=15,color='blue', orientation='horizontal');
ax_histx.tick_params(axis="x", labelbottom=False)
ax_histy.tick_params(axis="y", labelleft=False)
ax.tick_params(axis="y", labelright=False)

cbar = fig.colorbar(cax, label='[OII]7320 SNR')
plt.show()

### Plot [OIII] Temperatures

In [ ]:
x = teoiii
y = teoiii_cross

fig = plt.figure(figsize=(5, 5))

gs = fig.add_gridspec(2, 2,  width_ratios=(4, 1), height_ratios=(1, 4),
                      left=0.1, right=0.9, bottom=0.1, top=0.9,
                      wspace=0.0, hspace=0.0)

ax = fig.add_subplot(gs[1, 0])
ax_histx = fig.add_subplot(gs[0, 0], sharex=ax)
ax_histy = fig.add_subplot(gs[1, 1], sharey=ax)

# Cmap info
oii_where = np.where(~np.isnan(y))[0]

snr_7319 = np.array([data[i]['OII7319_FLUX_CORR_REFIT'][0]/data[i]['OII7319_FLUX_CORR_REFIT_ERR'][0] for i in range(len(data))])
snr_7330 = np.array([data[i]['OII7330_FLUX_CORR_REFIT'][0]/data[i]['OII7330_FLUX_CORR_REFIT_ERR'][0] for i in range(len(data))])

snr_7319 = snr_7319[oii_where]
snr_7330 = snr_7330[oii_where]

# the scatter plot
cax = ax.scatter(x[oii_where], y[oii_where], s=100, marker='.', c=snr_7319, cmap='bwr', edgecolors='k')
ax.set_xlim([7500, None])
ax.set_ylim([7500, None])
ax.plot(np.linspace(0, 10**6, 10), np.linspace(0, 10**6, 10), color='k')
#fig.suptitle('NII derived Temperature versus SII derived Density')
ax.set_xlabel('[OIII] Temperature [K] from getTemDen')
ax.set_ylabel('[OIII] Temperature [K] from getCrossTemDen')
#ax.set_yscale('log')

# the histograms
ax_histx.hist(x, bins=60, color='blue');
ax_histy.hist(y, bins=15,color='blue', orientation='horizontal');
ax_histx.tick_params(axis="x", labelbottom=False)
ax_histy.tick_params(axis="y", labelleft=False)
ax.tick_params(axis="y", labelright=False)

cbar = fig.colorbar(cax, label='[OII]7320 SNR')
plt.show()

# Make an emission grid with PyNeb

### Make the emission grid object for the [OII] ion

In [ ]:
O2_EG = pn.EmisGrid(elem='O', spec=2, n_tem=500, n_den=500, 
                    tem_min=6250., tem_max=7000., den_min=10., 
                    den_max=100, restore_file=None, atomObj=None)

### Get the emissivity grid of a combination of lines

In [ ]:
O2_Te = O2_EG.getGrid(to_eval='(L(3726)+L(3729))/(L(7319)+L(7320)+L(7330)+L(7331))')

### Plot the image of the emssion grid

In [ ]:
O2_EG.plotImage(to_eval='(L(3726)+L(3729))/(L(7319)+L(7320)+L(7330)+L(7331))')

### Plot the contours of the emission grid

In [ ]:
f, ax = plt.subplots(figsize=(6,5))
O2_EG.plotContours(to_eval='(L(3726)+L(3729))/(L(7319)+L(7320)+L(7330)+L(7331))', ax=ax)
#f.savefig('OIII_diag.pdf')

### Make a list of [OII] Temperature sensitive line ratios

In [ ]:
O2ratio_list = np.array([oii3727[i] / (data[i]['OII7319_FLUX_CORR_REFIT']+data[i]['OII7330_FLUX_CORR_REFIT']) for i in range(len(data)) if ~np.isnan(oii3727[i] / (data[i]['OII7319_FLUX_CORR_REFIT']+data[i]['OII7330_FLUX_CORR_REFIT']))])

### Make a histogram of [OII] flux ratios (logarithmic)

In [ ]:
plt.hist(np.log10(O2ratio_list), color = 'red', alpha=0.7, bins=25);
plt.axvline(np.median(np.log10(O2ratio_list)), color='k', linestyle='--', label=f'Median: {round(np.median(np.log10(O2ratio_list)), 3)}')
plt.xlabel(r'$log(\frac{[OII]3727}{[OII]7320 + [OII]7320})$')
plt.legend()

### Make a histogram of [OII] flux ratios

In [ ]:
plt.hist(O2ratio_list, color = 'red', alpha=0.7, bins=25);
plt.axvline(np.median(O2ratio_list), color='k', linestyle='--', label=f'Median: {round(np.median(O2ratio_list), 3)}')
plt.xlabel(r'$log(\frac{[OII]3727}{[OII]7320 + [OII]7320})$')
plt.legend()